In [19]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata, rrankdata
from pymcdm import weights as mcdm_weights

In [20]:
from prettytable import from_csv
with open("heart_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+----------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1-SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+----------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.8    |    0.86   |  0.8   | 0.949891068 |   0.8    | 0.352941176 |      0      | 0.647058824 | 0.196721311  | 6.794670687  |
|    DECISION TREE     |   0.74   |    0.76   |  0.74  | 0.749455338 |   0.74   | 0.352941176 | 0.148148148 | 0.647058824 | 0.262295082  | 9.059508483  |
| GAUSSIAN NAIVE BAYES |   0.87   |    0.9    |  0.87  | 0.934640523 |   0.87   | 0.235294118 |      0      | 0.764705882 | 0.131147541  | 4.529780458  |
|         SVM          |   0.44   |    0.2    |  0.44  | 0.566448802 |   0.2

In [21]:
x_matrix, alt_names = mcdm.load("heart_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weight = [0.10909090908971901,0.16363636363821488, 0.18181818182165294,0.12727272727352062,0.14545454545314052,0.0363636363625124,0.0727272727239339,0.0545454545446777,0.09090909091046283,0.018181818182165292]
weights=np.array(weight)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [22]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


2.0 0.8688923584178941
4.0 0.7293310983982733
1.0 0.9572522011247699
7.0 0.20321579498106554
6.0 0.26020687310738383
3.0 0.85906853200189
5.0 0.6373558152472117


In [23]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

2.0 0.10667655584933175
4.0 0.2686427536289488
1.0 0.0
7.0 1.0
6.0 0.7879668411601278
3.0 0.1199099334694391
5.0 0.3833103317534652


In [24]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.5454545454560191
4.0 -0.039393939392509625
1.0 0.9030303030317852
7.0 -0.8363636363689366
6.0 -0.5818181818205013
3.0 0.3969696969711846
5.0 -0.38787878787704133


In [25]:
from prettytable import PrettyTable
print("#########################PIMA DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################PIMA DATASET#######################
+----------------------+--------+---------------------+-------+---------------------+-----------+-----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |        VALUES       | PROMETHEE |         VALUES        |
+----------------------+--------+---------------------+-------+---------------------+-----------+-----------------------+
| LOGISTIC REGRESSION  |  2.0   |  0.8688923584178941 |  2.0  | 0.10667655584933175 |    2.0    |   0.5454545454560191  |
|    DECISION TREE     |  4.0   |  0.7293310983982733 |  4.0  |  0.2686427536289488 |    4.0    | -0.039393939392509625 |
| GAUSSIAN NAIVE BAYES |  1.0   |  0.9572522011247699 |  1.0  |         0.0         |    1.0    |   0.9030303030317852  |
|         SVM          |  7.0   | 0.20321579498106554 |  7.0  |         1.0         |    7.0    |  -0.8363636363689366  |
|    NEURAL NETWORK    |  6.0   | 0.26020687310738383 |  6.0  |  0.7879668411601278 |

In [26]:
ans=mcdm.rank(x_matrix, alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1],n_method="Linear2", w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
| GAUSSIAN NAIVE BAYES |  0.7742707157207465 |
| LOGISTIC REGRESSION  |  0.7185066460246889 |
|    RANDOM FOREST     |  0.7051037315605179 |
|    DECISION TREE     |  0.6245810239067179 |
|     LINEAR MODEL     |  0.5732921090172806 |
|    NEURAL NETWORK    | 0.34540593953354615 |
|         SVM          | 0.19052543793106042 |
+----------------------+---------------------+


In [27]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,n_method="Linear1",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    DECISION TREE     | 0.6628713135019321 |
|     LINEAR MODEL     | 0.6614412985215331 |
|    RANDOM FOREST     | 0.6552890781554467 |
| LOGISTIC REGRESSION  |        0.0         |
| GAUSSIAN NAIVE BAYES |        0.0         |
|         SVM          |        0.0         |
|    NEURAL NETWORK    |        0.0         |
+----------------------+--------------------+
